In [1]:
run startSpark.py

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.0
      /_/

Using Python version 2.7.11 (default, Jun 11 2016 11:33:47)
SparkSession available as 'spark'.


In [2]:
#load the rolling 6 months or 1 year from Redshift dvcstats.devie_page_count_all
theSQL = "(select SUBSTRING(account_code, 1,3) as acct_code,account_code,serial_number,reported_date,lifetime_page_count, color_lifetime_page_count,mono_lifetime_page_count,page_count,color_page_count,mono_page_count,time_dimension_surrogate_key, customer_master_surrogate_key,product_master_surrogate_key,material_master_surrogate_key,account_surrogate_key,asset_surrogate_key,agreement_surrogate_key,create_date_time,create_user from dvcstats.device_page_count_all where trunc(reported_date) >= current_date - INTERVAL '3 d') as myQuery"


databaseReader = (spark.read
                 .format("jdbc")
                 .option("url","jdbc:postgresql://duck.cff0iaonktcp.us-east-1.redshift.amazonaws.com:5439/idw?tcpKeepAlive=true&ssl=true&sslfactory=org.postgresql.ssl.NonValidatingFactory")
                 .option("dbtable",theSQL)
                 .option("user","pbishop")
                  .option("password","********$")
                  .option("driver","org.postgresql.Driver"))
#databaseReader
raw_df = databaseReader.load()
raw_df .cache()
raw_df .printSchema()

root
 |-- acct_code: string (nullable = true)
 |-- account_code: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- reported_date: timestamp (nullable = true)
 |-- lifetime_page_count: long (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- time_dimension_surrogate_key: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- product_master_surrogate_key: long (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- create_date_time: timestamp (nullable = true)
 |-- create_user: string (nullable = true)



In [4]:
raw_df.show()

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:62671)

In [3]:
#This gets the serial_numbers with negative page counts and aliases a couple columns

affectedSN = raw_df.select(raw_df.acct_code,raw_df.account_code,raw_df.serial_number,raw_df.reported_date.alias("neg_read_date"),raw_df.lifetime_page_count.alias("LTPC_Y"),raw_df.color_lifetime_page_count,raw_df.mono_lifetime_page_count,raw_df.page_count,raw_df.color_page_count,raw_df.mono_page_count,raw_df.time_dimension_surrogate_key,raw_df. customer_master_surrogate_key,raw_df.product_master_surrogate_key,raw_df.material_master_surrogate_key,raw_df.account_surrogate_key,raw_df.asset_surrogate_key,raw_df.agreement_surrogate_key,raw_df.create_date_time,raw_df.create_user).filter("page_count < 0")
affectedSN.cache()
affectedSN.printSchema()

root
 |-- acct_code: string (nullable = true)
 |-- account_code: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- neg_read_date: timestamp (nullable = true)
 |-- LTPC_Y: long (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- time_dimension_surrogate_key: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- product_master_surrogate_key: long (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- create_date_time: timestamp (nullable = true)
 |-- create_user: string (nullable = true)



In [3]:
patt1 = spark.read.json("./Output 2017-03-17-23-20-14-UTC")
patt1.cache()
patt1.printSchema()
patt1.count()

root
 |-- account_code: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- create_date_time: string (nullable = true)
 |-- create_user: string (nullable = true)
 |-- curr_page_count: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- ltpc_x: long (nullable = true)
 |-- ltpc_x_date: string (nullable = true)
 |-- ltpc_y: long (nullable = true)
 |-- ltpc_z: long (nullable = true)
 |-- ltpc_z_date: string (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- neg_read_date: string (nullable = true)
 |-- page_count: long (nullable = true)
 |-- product_master_surrogate_key: string (nullable = t

3995

In [5]:
upsr = spark.read.text("./UPSRECEIVE S3 Files")
ups = upsr.map(lambda l: l.split("\t"))
p7 = ups.map(lambda p: Row(serial_number=p[0].replace("\"", ""),del_date=p[1].replace("\"", ""),lifetime_page_count=p[2].replace("\"", ""),printer_name=p[3].replace("\"", ""),ip_address=p[4].replace("\"", ""),account_code=p[5].replace("\"", ""),color_lpc=p[6].replace("\"", ""),mono_lpc=p[7].replace("\"", ""),fax_lpc=p[8].replace("\"", ""),scan_lpc=p[9].replace("\"", ""),color_copy_lpc=p[10].replace("\"", ""),mono_copy_lpc=p[11].replace("\"", ""),total_copy_lpc=p[12].replace("\"", ""),suspect_state=p[13].replace("\"", ""),data_source=p[14].replace("\"", "")))
schemaP7 = spark.createDataFrame(p7)
schemaP7.createOrReplaceTempView("ups")

AttributeError: 'DataFrame' object has no attribute 'map'

In [11]:
input = "./UPSRECEIVE S3 Files"
upsr = spark.read.csv(input, sep='\t', header='true')
upsr.show()

+-------------+-------------------+---------------+-------------+---------------+---------+---------+--------+-------+--------+--------------+-------------+--------------+-------------+-----------+
|SERIAL_NUMBER|           DEL_DATE|LIFE_PAGE_COUNT| PRINTER_NAME|     IP_ADDRESS|ACCT_CODE|COLOR_LPC|MONO_LPC|FAX_LPC|SCAN_LPC|COLOR_COPY_LPC|MONO_COPY_LPC|TOTAL_COPY_LPC|SUSPECT_STATE|DATA_SOURCE|
+-------------+-------------------+---------------+-------------+---------------+---------+---------+--------+-------+--------+--------------+-------------+--------------+-------------+-----------+
|74635C6601ZM7|2017-03-16 20:19:38|          79117|Lexmark MX711| 180.170.159.13|    SAN01|        0|       0|      0|       0|             0|            0|             0|            1|          6|
|74635C6601ZMY|2017-03-16 20:19:38|           9905|Lexmark MX711|  10.155.213.54|    SAN01|        0|       0|      0|       0|             0|            0|             0|            1|          6|
|74635C660

In [13]:
upsr.createOrReplaceTempView("upsr")

In [15]:
spark.sql("select acct_code, serial_number, del_date, life_page_count from upsr").show()


+---------+-------------+-------------------+---------------+
|acct_code|serial_number|           del_date|life_page_count|
+---------+-------------+-------------------+---------------+
|    SAN01|74635C6601ZM7|2017-03-16 20:19:38|          79117|
|    SAN01|74635C6601ZMY|2017-03-16 20:19:38|           9905|
|    SAN01|74635C6601ZLH|2017-03-16 20:19:38|         120956|
|    SAN01|74635C6601ZK0|2017-03-16 20:19:38|          90389|
|    SAN01|74635C6601ZKG|2017-03-16 20:19:38|          96745|
|    SAN01|74635C6601Y2K|2017-03-16 20:19:38|          20952|
|    SAN01|74635C6600T6N|2017-03-16 20:19:44|          25760|
|    SAN01|74635C6600T5G|2017-03-16 20:19:44|          59779|
|    SAN01|74635C6600T40|2017-03-16 20:19:44|          56068|
|    SAN01|74635C6600T27|2017-03-16 20:19:44|         193458|
|    SAN01|74635C6600T2H|2017-03-16 20:19:44|          26484|
|    SAN01|74635C6600T1Z|2017-03-16 20:19:44|          20626|
|    SAN01|74635C6600T1H|2017-03-16 20:19:44|           3628|
|    SAN

In [16]:
spark.sql("select max(del_date) from upsr").show()

+-------------------+
|      max(del_date)|
+-------------------+
|2017-03-16 23:59:53|
+-------------------+



In [19]:
patt2 = spark.read.json("./Output 2017-03-20-20-49-08-UTC/")
patt2.cache()
patt2.printSchema()
patt2.count()

root
 |-- account_code: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- create_date_time: string (nullable = true)
 |-- create_user: string (nullable = true)
 |-- curr_page_count: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- ltpc_x: long (nullable = true)
 |-- ltpc_x_date: string (nullable = true)
 |-- ltpc_y: long (nullable = true)
 |-- ltpc_z: long (nullable = true)
 |-- ltpc_z_date: string (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- neg_read_date: string (nullable = true)
 |-- page_count: long (nullable = true)
 |-- product_master_surrogate_key: string (nullable = t

4044

In [20]:
patt2.createOrReplaceTempView("patt2")

In [21]:
spark.sql("select account_code, serial_number, neg_read_date, ltpc_y, page_count, curr_page_count from patt2").show()

+------------+-------------+-------------------+------+----------+---------------+
|account_code|serial_number|      neg_read_date|ltpc_y|page_count|curr_page_count|
+------------+-------------+-------------------+------+----------+---------------+
|       LOT01|      0000000|2016-04-01 17:40:24| 47602|    -71330|             30|
|       AHS01|   0000000000|2017-02-07 23:07:29| 55610|        -2|             25|
|       KPT02|      0252657|2017-02-15 04:14:52|   267|   -851033|            558|
|       BSF02|      0435299|2016-05-24 20:54:35|662248|    -96737|            230|
|       UBU14|      0471198|2016-10-19 23:40:11| 57739|   -809112|             27|
|       BSF01|      0471572|2016-02-29 13:21:52|     2|    -48229|             47|
|       BNS01|      0611882|2017-03-01 20:52:04|    11|   -448658|              3|
|       DEL01|      0622972|2016-03-25 13:41:47|137034|   -105805|            130|
|       KPT02|      0631491|2016-06-09 20:37:56|    84|   -351314|              0|
|   

In [22]:
patt3 = spark.read.json("./Output 2017-03-21-14-03-41-UTC/")
patt3.cache()
patt3.printSchema()
patt3.count()

root
 |-- account_code: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- create_date_time: string (nullable = true)
 |-- create_user: string (nullable = true)
 |-- curr_page_count: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- ltpc_x: long (nullable = true)
 |-- ltpc_x_date: string (nullable = true)
 |-- ltpc_y: long (nullable = true)
 |-- ltpc_z: long (nullable = true)
 |-- ltpc_z_date: string (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- neg_page_count: long (nullable = true)
 |-- neg_read_date: string (nullable = true)
 |-- product_master_surrogate_key: string (nullable

3405

In [23]:
patt3.createOrReplaceTempView("pattn3")

In [27]:
spark.sql("select account_code, serial_number, neg_read_date, ltpc_y, neg_page_count, curr_page_count,create_date_time from pattn3").show()

+------------+-------------+-------------------+------+--------------+---------------+-------------------+
|account_code|serial_number|      neg_read_date|ltpc_y|neg_page_count|curr_page_count|   create_date_time|
+------------+-------------+-------------------+------+--------------+---------------+-------------------+
|       LOT01|      0000000|2016-04-01 17:40:24| 47602|        -71330|             30|2017-03-21 15:20:52|
|       BSF02|      0435299|2016-05-24 20:54:35|662248|        -96737|            230|2017-03-21 15:20:52|
|       BSF01|      0471572|2016-02-29 13:21:52|     2|        -48229|             47|2017-03-21 15:20:52|
|       BNS01|      0611882|2017-03-01 20:52:04|    11|       -448520|              0|2017-03-21 15:20:52|
|       DEL01|      0622972|2016-03-25 13:41:47|137034|       -105805|              0|2017-03-21 15:20:52|
|       KPT02|      0631491|2016-06-09 20:37:56|    84|       -351314|              0|2017-03-21 15:20:52|
|       HFO02|      0650375|2016-04-1

In [28]:
patt4 = spark.read.json("./Output 2017-03-21-15-21-24-UTC/")
patt4.cache()
patt4.printSchema()
patt4.count()

root
 |-- account_code: string (nullable = true)
 |-- account_surrogate_key: string (nullable = true)
 |-- agreement_surrogate_key: string (nullable = true)
 |-- asset_surrogate_key: string (nullable = true)
 |-- color_lifetime_page_count: long (nullable = true)
 |-- color_page_count: long (nullable = true)
 |-- create_date_time: string (nullable = true)
 |-- create_user: string (nullable = true)
 |-- curr_page_count: long (nullable = true)
 |-- customer_master_surrogate_key: string (nullable = true)
 |-- ltpc_x: long (nullable = true)
 |-- ltpc_x_date: string (nullable = true)
 |-- ltpc_y: long (nullable = true)
 |-- ltpc_z: long (nullable = true)
 |-- ltpc_z_date: string (nullable = true)
 |-- material_master_surrogate_key: string (nullable = true)
 |-- mono_lifetime_page_count: long (nullable = true)
 |-- mono_page_count: long (nullable = true)
 |-- neg_read_date: string (nullable = true)
 |-- page_count: long (nullable = true)
 |-- product_master_surrogate_key: string (nullable = t

4051

In [29]:
patt4.createOrReplaceTempView("pattn4")

In [31]:
spark.sql("select account_code, serial_number, neg_read_date, ltpc_y, page_count, curr_page_count,create_date_time from pattn4").show()

+------------+-------------+-------------------+------+----------+---------------+-------------------+
|account_code|serial_number|      neg_read_date|ltpc_y|page_count|curr_page_count|   create_date_time|
+------------+-------------+-------------------+------+----------+---------------+-------------------+
|       LOT01|      0000000|2016-04-01 17:40:24| 47602|    -71330|             30|2017-03-21 16:20:46|
|       AHS01|   0000000000|2017-02-07 23:07:29| 55610|        -2|              0|2017-03-21 16:20:46|
|       KPT02|      0252657|2017-02-15 04:14:52|   267|   -851033|            756|2017-03-21 16:20:46|
|       BSF02|      0435299|2016-05-24 20:54:35|662248|    -96737|            230|2017-03-21 16:20:46|
|       UBU14|      0471198|2016-10-19 23:40:11| 57739|   -809112|              0|2017-03-21 16:20:46|
|       BSF01|      0471572|2016-02-29 13:21:52|     2|    -48229|             47|2017-03-21 16:20:46|
|       BNS01|      0611882|2017-03-01 20:52:04|    11|   -448658|       

In [42]:
input = "./test-ups-input"
#upsr = spark.read.csv(input, sep='\t', header='true')
ups = spark.read.csv(input, sep='\t')
ups.show()

+-------------+-------------------+------+-------------+--------------+-----+-----+----+---+---+----+----+----+----+----+
|          _c0|                _c1|   _c2|          _c3|           _c4|  _c5|  _c6| _c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|
+-------------+-------------------+------+-------------+--------------+-----+-----+----+---+---+----+----+----+----+----+
|7526159441C8P|2015-01-01 03:27:08| 16977| Lexmark X748|    10.220.5.9|ADB01|14197|2780|  0|  0|   0|   0|   0|   0|   6|
|74633699055WN|2015-01-01 14:45:19|333997|Lexmark MX810|   10.21.5.183|AHS01|    0|   0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2015-01-01 14:45:19| 21854|Lexmark MX911|   10.2.200.32|AHS01|    0|   0|  0|  0|   0|   0|   0|   1|   6|
|406347990M7MK|2015-01-01 16:32:19| 18047|Lexmark MS811|172.20.159.117|AAR01|    0|   0|  0|  0|   0|   0|   0|   0|   6|
|7526159441C8P|2015-01-02 03:48:19| 17034| Lexmark X748|    10.220.5.9|ADB01|14251|2783|  0|  0|   0|   0|   0|   0|   6|
|74633699055WN|2015-01-0

In [45]:
ups.count()

1938

In [43]:
ups.createOrReplaceTempView("ups")

In [44]:
spark.sql("select * from ups order by _c0, _c1").show()

+-------------+-------------------+----+-------------+-------------+-----+---+---+---+---+----+----+----+----+----+
|          _c0|                _c1| _c2|          _c3|          _c4|  _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|
+-------------+-------------------+----+-------------+-------------+-----+---+---+---+---+----+----+----+----+----+
|406347990M7MK|2014-10-17 15:24:49| 242|Lexmark MS811|172.28.68.189|AAR01|  0|  0|  0|  0|   0|   0|   0|   0|   6|
|406347990M7MK|2014-10-18 15:36:15| 560|Lexmark MS811|172.28.68.189|AAR01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|406347990M7MK|2014-10-19 15:46:29| 688|Lexmark MS811|172.28.68.189|AAR01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|406347990M7MK|2014-10-20 15:57:35| 710|Lexmark MS811|172.28.68.189|AAR01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|406347990M7MK|2014-10-21 16:08:58| 918|Lexmark MS811|172.28.68.189|AAR01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|406347990M7MK|2014-10-22 16:20:20|1117|Lexmark MS811|172.28.68.189|AAR0

In [46]:
spark.sql("select _c0, min(_c1) min_dt, max(_c1) max_dt, count(*) from ups group by _c0").show()

+-------------+-------------------+-------------------+--------+
|          _c0|             min_dt|             max_dt|count(1)|
+-------------+-------------------+-------------------+--------+
|406347990M7MK|2014-10-17 15:24:49|2015-12-31 20:52:00|     392|
|7526159441C8P|2014-06-19 21:00:52|2015-12-31 22:32:46|     550|
|74633699055WN|2014-04-03 09:29:51|2015-12-31 01:40:54|     591|
|7421013000575|2014-11-07 05:55:59|2015-12-31 01:40:53|     405|
+-------------+-------------------+-------------------+--------+



In [48]:
input = "./test-ups-input-2"
#upsr = spark.read.csv(input, sep='\t', header='true')
ups1 = spark.read.csv(input, sep='\t')
ups1.show()

+-------------+-------------------+------+-------------+-----------+-----+---+---+---+---+----+----+----+----+----+
|          _c0|                _c1|   _c2|          _c3|        _c4|  _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|
+-------------+-------------------+------+-------------+-----------+-----+---+---+---+---+----+----+----+----+----+
|7421013000575|2016-11-24 11:43:51|469538|Lexmark MX911|10.2.200.32|AHS01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2016-11-25 12:05:10|473175|Lexmark MX911|10.2.200.32|AHS01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2016-11-26 12:26:27|473520|Lexmark MX911|10.2.200.32|AHS01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2016-11-27 12:48:34|473520|Lexmark MX911|10.2.200.32|AHS01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2016-11-28 13:07:59|477832|Lexmark MX911|10.2.200.32|AHS01|  0|  0|  0|  0|   0|   0|   0|   1|   6|
|7421013000575|2016-11-29 13:29:15|481072|Lexmark MX911|10.2.200.32|AHS0

In [49]:
ups1.createOrReplaceTempView("ups1")

In [50]:
spark.sql("select _c0, min(_c1) min_dt, max(_c1) max_dt, count(*) from ups1 group by _c0").show()

+-------------+-------------------+-------------------+--------+
|          _c0|             min_dt|             max_dt|count(1)|
+-------------+-------------------+-------------------+--------+
|406347990M7MK|2014-10-17 15:24:49|2017-03-20 23:21:38|     811|
|7526159441C8P|2014-06-19 21:00:52|2017-03-21 00:30:32|     977|
|74633699055WN|2014-04-03 09:29:51|2017-03-21 09:20:53|    1019|
|7421013000575|2014-11-07 05:55:59|2017-03-21 09:20:53|     772|
+-------------+-------------------+-------------------+--------+

